In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()  # reads .env
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [16]:
def score_tone(speech_text):
    prompt = f"""
    You are a financial policy analyst. Given the Federal Reserve speech below,
    rate the speaker’s tone toward each of the following topics on a continuous scale
    from -1 (dovish) to +1 (hawkish). Return only six numbers in JSON format
    with keys exactly as shown below.

    Topics and scoring guide:
    - Fed Funds Rate: hawkish = likely rate hike; dovish = likely rate cut
    - Labor Market: hawkish = strong labor market; dovish = weak labor market
    - Inflation: hawkish = high or concerning inflation; dovish = low or less concerning inflation
    - Real Activity: hawkish = strong real activity; dovish = weak real activity
    - Financial Stability: hawkish = stable system; dovish = less stable system
    - Balance Sheet: hawkish = in or start of QT or balance sheet reduction, end of QE; dovish = start or in QE or expansion, end of QT

    Respond ONLY with JSON like this:
    {{
      "Fed Funds Rate": 0.4,
      "Labor Market": -0.2,
      "Inflation": 0.6,
      "Real Activity": 0.1,
      "Financial Stability": 0.3,
      "Balance Sheet": -0.5
    }}

    --- Speech ---
    {speech_text}
    """

    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content


In [34]:
author_key = "williams"

In [35]:

def json_by_id(select_id):
    with open(f"text_data/{author_key}.json", "r", encoding="utf-8") as f:
        text_data = json.load(f)
        selected_text = list(filter(lambda x: x["id"]==select_id, text_data))
        return selected_text[0]

def read_all_ids():
    with open(f"text_data/{author_key}.json", "r", encoding="utf-8") as f:
        text_data    = json.load(f)
        selected_ids = list(map(lambda x: x["id"], text_data))
        return selected_ids

In [36]:
ids = read_all_ids()
#ids = ["fedgsq:101935"]
data = []
for text_id in ids:
    json_result = json_by_id(text_id)
    tone_score = score_tone(json_result["text"])
    tone_score = json.loads(tone_score)
    result = {"id": text_id}
    result["gpt-5"] = tone_score
    print(result)
    data.append(result)

with open(f"info_folder/score_{author_key}.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)


{'id': 'fednsp:30108', 'gpt-5': {'Fed Funds Rate': 0.0, 'Labor Market': -0.3, 'Inflation': 0.0, 'Real Activity': -0.2, 'Financial Stability': 0.0, 'Balance Sheet': 0.0}}
{'id': 'fednsp:87592', 'gpt-5': {'Fed Funds Rate': -0.8, 'Labor Market': 0.5, 'Inflation': -0.4, 'Real Activity': -0.6, 'Financial Stability': 0.1, 'Balance Sheet': -0.8}}
{'id': 'fednsp:102070', 'gpt-5': {'Fed Funds Rate': 0.0, 'Labor Market': 0.0, 'Inflation': 0.0, 'Real Activity': 0.0, 'Financial Stability': 0.5, 'Balance Sheet': -0.8}}
{'id': 'fednsp:95493', 'gpt-5': {'Fed Funds Rate': 0.85, 'Labor Market': 0.7, 'Inflation': 0.75, 'Real Activity': -0.35, 'Financial Stability': 0.4, 'Balance Sheet': 0.9}}
{'id': 'fednsp:98774', 'gpt-5': {'Fed Funds Rate': -0.85, 'Labor Market': -0.2, 'Inflation': -0.6, 'Real Activity': 0.2, 'Financial Stability': 0.4, 'Balance Sheet': 0.2}}
{'id': 'fednsp:88032', 'gpt-5': {'Fed Funds Rate': -0.9, 'Labor Market': -0.95, 'Inflation': -0.6, 'Real Activity': -0.85, 'Financial Stability'